In [ ]:
from dash import Dash, html, Input, Output, callback_context  # Asegúrate de importar callback_context
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [150]:
file_path="sindummieslimpio.csv"

df=pd.read_csv(file_path)

unicos={}

for i in df.columns:
    unicos[i]=df[i].unique()

unicos.keys()

dict_keys(['periodo', 'cole_area_ubicacion', 'cole_bilingue', 'cole_calendario', 'cole_caracter', 'cole_jornada', 'cole_naturaleza', 'estu_genero', 'estu_nacionalidad', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda', 'fami_personashogar', 'fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tienelavadora', 'punt_ingles', 'punt_matematicas', 'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica', 'punt_global', 'estu_edad'])

In [151]:
unicos['fami_tienecomputador']

array(['Desconocido', 'Si', 'No'], dtype=object)

In [178]:


#Lanzar app
app = Dash(__name__, external_stylesheets=[dbc.themes.SOLAR], suppress_callback_exceptions=True)


def tab1_layout():
    return dbc.Container([
        html.Br(),
        # Título
        dbc.Row([
            dbc.Col(html.H3("Entrada de Datos", className="text-light mb-7 text-center"), width=12)
        ]),

        dbc.Row([
            dbc.Col([
                html.Label("Edad del estudiante:", className="text-success mb-2 text-center"),
                dcc.Slider(
                    id="slider-edad",
                    min=0,
                    max=100,
                    step=1,
                    value=10,
                    marks={i: str(i) for i in range(0, 101, 5)},
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ], width=12)
        ]),


        # Contenido dividido en dos columnas
        dbc.Row([
            # Primera columna (8 categorías)
            dbc.Col([
                html.Label("Área de ubicación del colegio:", className="text-primary mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-ubicacion",
                    options=[
                        {"label": "URBANO", "value": "URBANO"},
                        {"label": "RURAL", "value": "RURAL"}
                    ],
                    value="URBANO",
                    placeholder="Selecciona una opción...",
                ),
                html.Br(),

                html.Label("¿El colegio es bilingüe?", className="text-primary mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-bilingue",
                    options=[
                        {"label": "No", "value": "N"},
                        {"label": "Desconocido", "value": "Desconocido"},
                        {"label": "Sí", "value": "S"}
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Calendario del colegio:", className="text-primary mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-calendario",
                    options=[
                        {"label": "A", "value": "A"},
                        {"label": "B", "value": "B"}
                    ],
                    value="A",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Carácter del colegio:", className="text-primary mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-caracter",
                    options=[
                        {"label": "ACADÉMICO", "value": "ACADÉMICO"},
                        {"label": "TÉCNICO", "value": "TÉCNICO"},
                        {"label": "TÉCNICO/ACADÉMICO", "value": "TÉCNICO/ACADÉMICO"},
                        {"label": "Desconocido", "value": "Desconocido"},
                        {"label": "NO APLICA", "value": "NO APLICA"}
                    ],
                    value="ACADÉMICO",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Jornada del colegio:", className="text-primary mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-jornada",
                    options=[
                        {"label": "ÚNICA", "value": "ÚNICA"},
                        {"label": "TARDE", "value": "TARDE"},
                        {"label": "NOCHE", "value": "NOCHE"},
                        {"label": "MAÑANA", "value": "MAÑANA"},
                        {"label": "COMPLETA", "value": "COMPLETA"},
                        {"label": "SABATINA", "value": "SABATINA"}
                    ],
                    value="ÚNICA",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Naturaleza del colegio:", className="text-primary mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-naturaleza",
                    options=[
                        {"label": "OFICIAL", "value": "OFICIAL"},
                        {"label": "NO OFICIAL", "value": "NO OFICIAL"}
                    ],
                    value="OFICIAL",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Selecciona el género del estudiante:", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-genero",
                    options=[
                        {"label": "Femenino", "value": "F"},
                        {"label": "Masculino", "value": "M"},
                        {"label": "Desconocido", "value": "Desconocido"}
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Selecciona la nacionalidad del estudiante:", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-nacionalidad",
                    options=[
                        {"label": "Colombia", "value": "COLOMBIA"},
                        {"label": "Venezuela", "value": "VENEZUELA"},
                        {"label": "España", "value": "ESPAÑA"},
                        {"label": "Ecuador", "value": "ECUADOR"},
                        {"label": "Guatemala", "value": "GUATEMALA"}
                    ],
                    value="COLOMBIA",
                    placeholder="Selecciona una opción..."
                ),
            ], width=6),

            # Segunda columna (8 categorías)
            dbc.Col([
                html.Label("Nivel educativo de la madre:", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-educacionmadre",
                    options=[
                        {"label": nivel, "value": nivel} for nivel in [
                            'Desconocido', 'Secundaria (Bachillerato) completa', 'Primaria incompleta',
                            'Educación profesional completa', 'Técnica o tecnológica completa',
                            'Secundaria (Bachillerato) incompleta', 'Primaria completa', 'No sabe',
                            'Ninguno', 'Educación profesional incompleta', 'Postgrado', 'No Aplica',
                            'Técnica o tecnológica incompleta'
                        ]
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Nivel educativo del padre:", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-educacionpadre",
                    options=[
                        {"label": nivel, "value": nivel} for nivel in [
                            'Desconocido', 'Secundaria (Bachillerato) completa', 'Primaria incompleta',
                            'Educación profesional completa', 'Técnica o tecnológica completa',
                            'Secundaria (Bachillerato) incompleta', 'Primaria completa', 'No sabe',
                            'Ninguno', 'Educación profesional incompleta', 'Postgrado', 'No Aplica',
                            'Técnica o tecnológica incompleta'
                        ]
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Estrato de la vivienda:", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-estratovivienda",
                    options=[
                        {"label": estrato, "value": estrato} for estrato in [
                            'Desconocido', 'Estrato 1', 'Estrato 2', 'Estrato 3',
                            'Estrato 4', 'Estrato 5', 'Estrato 6', 'Sin Estrato'
                        ]
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("Número de personas en el hogar:", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-personashogar",
                    options=[
                        {"label": numero, "value": numero} for numero in [
                            'Desconocido', '1 a 2', '3 a 4', '5 a 6', '7 a 8',
                            '9 o más', 'Una', 'Dos', 'Tres', 'Cuatro', 'Cinco',
                            'Seis', 'Siete', 'Ocho', 'Nueve', 'Diez', 'Once',
                            'Doce o más'
                        ]
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("¿La familia tiene automóvil?", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-tieneautomovil",
                    options=[
                        {"label": "Desconocido", "value": "Desconocido"},
                        {"label": "Sí", "value": "Si"},
                        {"label": "No", "value": "No"}
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("¿La familia tiene computador?", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-tienecomputador",
                    options=[
                        {"label": "Desconocido", "value": "Desconocido"},
                        {"label": "Sí", "value": "Si"},
                        {"label": "No", "value": "No"}
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción..."
                ),
                html.Br(),

                html.Label("¿La familia tiene internet?", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-tieneinternet",
                    options=[
                        {"label": "Desconocido", "value": "Desconocido"},
                        {"label": "Sí", "value": "Si"},
                        {"label": "No", "value": "No"}
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción...",
                    style={
                    "backgroundColor": "#f0f0f0",  # Cambia el color de fondo
                    "color": "#000000",           # Cambia el color del texto
                    "border": "1px solid #ccc",   # Cambia el estilo del borde
                    "borderRadius": "5px"         # Añade bordes redondeados
                    }
                ),
                html.Br(),

                html.Label("¿La familia tiene lavadora?", className="text-success mb-2 text-center"),
                dcc.Dropdown(
                    id="dropdown-tienelavadora",
                    options=[
                        {"label": "Desconocido", "value": "Desconocido"},
                        {"label": "Sí", "value": "Si"},
                        {"label": "No", "value": "No"}
                    ],
                    value="Desconocido",
                    placeholder="Selecciona una opción...",
                ),
            ], width=6),
        ]),
    ], fluid=True)


def tab2_layout():
    return dbc.Container([
        html.Br(),
        # Primera fila: Predicción de Puntaje
        dbc.Row([
            dbc.Col([
                html.Br(),
                html.H3("Predicción de Puntaje:", className="text-light mb-2 text-center",
                        style={
                        "fontSize": "80px",  # Tamaño del texto
                        "textAlign": "center"  # Centrar el texto
                    }),
                html.Br(),
                html.Br(),
                html.H1(
                    id="contador-puntaje",
                    className="display-3 text-center",
                    style={
                        "fontSize": "120px",  # Tamaño del texto
                        "textAlign": "center"  # Centrar el texto
                    }  # Color se controlará dinámicamente
                ),  # Número dinámico
            ], width=12)
        ]),

        # Intervalo para el efecto del contador
        dcc.Interval(
            id="intervalo-contador",
            interval=30,  # Actualiza cada 30ms
            n_intervals=0,  # Comienza desde 0
            max_intervals=100  # Número máximo de iteraciones
        )
    ], fluid=True)


@app.callback(
    Output("contador-puntaje", "children"),
    Output("contador-puntaje", "style"),
    Input("intervalo-contador", "n_intervals")
)
def actualizar_puntaje(n_intervals):
    # Lógica del contador
    valor_final = 465  # Número final deseado
    valor_actual = int((n_intervals / 100) * valor_final)  # Incremento proporcional
    if n_intervals >= 100:
        valor_actual = valor_final  # Detener en el valor final

    # Cambiar color basado en el valor actual
    if valor_actual <= 200:
        color = "red"
    elif 201 <= valor_actual <= 400:
        color = "orange"
    else:
        color = "green"

    # Devolver valor y estilo
    return f"{valor_actual:,}", {
        "fontSize": "120px",  # Tamaño del texto
        "textAlign": "center",  # Centrar el texto
        "color": color  # Cambiar el color dinámicamente
    }



#Distribución de la app
app.layout=dbc.Container([

    #Titulo
    dbc.Row([
        dbc.Col(html.H1("Puntaje prueba ICFES", className="text-center text-light mb-4"), width=12)
    ]),

    #Pestañas: Introducir datos y visualizar resultados
    dbc.Row([
        dbc.Col(
            dcc.Tabs(id="tabs", value="tab1", children=[
                dcc.Tab(label="Entrada de datos", value="tab1", className="custom-tab"),
                dcc.Tab(label="Resultados", value="tab2", className="custom-tab")
            ], className="custom-tabs-container"), width=12
        )

    ]),

    # Contenedor para el contenido dinámico de las pestañas
    dbc.Row([
        dbc.Col(html.Div(id="tabs-content"), width=12)
    ])


], fluid=True)

@app.callback(
    Output("tabs-content", "children"),
    Input("tabs", "value")
)
def render_tab_content(tab):
    if tab == "tab1":
        return tab1_layout()
    elif tab == "tab2":
        return tab2_layout()
    

@app.callback(
    Output("output-dummies", "children"),
    [
        Input("dropdown-ubicacion", "value"),
        Input("dropdown-bilingue", "value"),
        Input("dropdown-calendario", "value"),
        Input("dropdown-caracter", "value"),
        Input("dropdown-jornada", "value"),
        Input("dropdown-naturaleza", "value"),
        Input("dropdown-genero", "value"),
        Input("dropdown-nacionalidad", "value"),
        Input("dropdown-educacionmadre", "value"),
        Input("dropdown-educacionpadre", "value"),
        Input("dropdown-estratovivienda", "value"),
        Input("dropdown-personashogar", "value"),
        Input("dropdown-tieneautomovil", "value"),
        Input("dropdown-tienecomputador", "value"),
        Input("dropdown-tieneinternet", "value"),
        Input("dropdown-tienelavadora", "value"),
    ]
)
def actualizar_dummies(ubicacion, bilingue, calendario, caracter, jornada, naturaleza, genero, nacionalidad,
                       educacionmadre, educacionpadre, estratovivienda, personashogar, tieneautomovil,
                       tienecomputador, tieneinternet, tienelavadora):
    # Generar variables dummy
    dummies = {
        # Ubicación
        "cole_area_ubicacion_URBANO": 1 if ubicacion == "URBANO" else 0,
        #"cole_area_ubicacion_RURAL": 1 if ubicacion == "RURAL" else 0,

        # Colegio bilingüe
        "cole_bilingue_N": 1 if bilingue == "N" else 0,
        "cole_bilingue_S": 1 if bilingue == "S" else 0,
        #"cole_bilingue_Desconocido": 1 if bilingue == "Desconocido" else 0,

        # Calendario
        #"cole_calendario_A": 1 if calendario == "A" else 0,
        "cole_calendario_B": 1 if calendario == "B" else 0,

        # Carácter del colegio
        #"cole_caracter_ACADEMICO": 1 if caracter == "ACADÉMICO" else 0,
        "cole_caracter_Desconocido": 1 if caracter == "Desconocido" else 0,
        "cole_caracter_NO_APLICA": 1 if caracter == "NO APLICA" else 0,
        "cole_caracter_TECNICO": 1 if caracter == "TÉCNICO" else 0,
        "cole_caracter_TECNICO_ACADEMICO": 1 if caracter == "TÉCNICO/ACADÉMICO" else 0,
        
        

        # Jornada del colegio
        "cole_jornada_MANIANA": 1 if jornada == "MAÑANA" else 0,
        "cole_jornada_NOCHE": 1 if jornada == "NOCHE" else 0,
        "cole_jornada_SABATINA": 1 if jornada == "SABATINA" else 0,
        "cole_jornada_TARDE": 1 if jornada == "TARDE" else 0,
        "cole_jornada_UNICA": 1 if jornada == "ÚNICA" else 0,
        #"cole_jornada_COMPLETA": 1 if jornada == "COMPLETA" else 0,
        

        # Naturaleza del colegio
        "cole_naturaleza_OFICIAL": 1 if naturaleza == "OFICIAL" else 0,
        #"cole_naturaleza_NO_OFICIAL": 1 if naturaleza == "NO OFICIAL" else 0,

        # Género del estudiante
        "genero_F": 1 if genero == "F" else 0,
        "genero_M": 1 if genero == "M" else 0,
        #"genero_Desconocido": 1 if genero == "Desconocido" else 0,

        # Nacionalidad del estudiante
        #"nacionalidad_COLOMBIA": 1 if nacionalidad == "COLOMBIA" else 0,
        "nacionalidad_ECUADOR": 1 if nacionalidad == "ECUADOR" else 0,
        "nacionalidad_ESPANIA": 1 if nacionalidad == "ESPAÑA" else 0,
        "nacionalidad_GUATEMALA": 1 if nacionalidad == "GUATEMALA" else 0,
        "nacionalidad_VENEZUELA": 1 if nacionalidad == "VENEZUELA" else 0,
    

        # Nivel educativo de la madre  SE QUITÓ DESCONOCIDO
        **{f"educacion_madre_{nivel}": 1 if educacionmadre == nivel else 0 for nivel in [
            "Educación profesional completa", "Educación profesional incompleta", "Ninguno", "No Aplica",
            "No sabe", "Postgrado", "Primaria completa", "Primaria incompleta","Secundaria (Bachillerato) completa",
            "Secundaria (Bachillerato) incompleta", "Técnica o tecnológica completa", "Técnica o tecnológica incompleta"
        ]},



        # Nivel educativo del padre SE QUITÓ DESCONOCIDO
        **{f"educacion_padre_{nivel}": 1 if educacionpadre == nivel else 0 for nivel in [
            "Educación profesional completa", "Educación profesional incompleta", "Ninguno", "No Aplica",
            "No sabe", "Postgrado", "Primaria completa", "Primaria incompleta","Secundaria (Bachillerato) completa",
            "Secundaria (Bachillerato) incompleta", "Técnica o tecnológica completa", "Técnica o tecnológica incompleta"
        ]},

        # Estrato de la vivienda SE QUITÓ DESCONOCIDO
        **{f"estrato_vivienda_{estrato}": 1 if estratovivienda == estrato else 0 for estrato in [
            "Estrato 1", "Estrato 2", "Estrato 3", "Estrato 4", "Estrato 5", "Estrato 6", "Sin Estrato"
        ]},

        # Número de personas en el hogar SE QUITO "1 a 2"
        **{f"personas_hogar_{personas}": 1 if personashogar == personas else 0 for personas in [
             "3 a 4", "5 a 6", "7 a 8", "9 o más","Cinco", "Cuatro", "Desconocido", "Diez", "Doce o más", 
             "Dos", "Nueve", "Ocho", "Once", "Seis","Siete","Tres", "Una"
     
        ]},

        # Automóvil
        "familia_tiene_automovil_No": 1 if tieneautomovil == "No" else 0,
        "familia_tiene_automovil_Si": 1 if tieneautomovil == "Si" else 0,
        #"familia_tiene_automovil_Desconocido": 1 if tieneautomovil == "Desconocido" else 0,

        # Computador
        "familia_tiene_computador_No": 1 if tienecomputador == "No" else 0,
        "familia_tiene_computador_Si": 1 if tienecomputador == "Si" else 0,
        #"familia_tiene_computador_Desconocido": 1 if tienecomputador == "Desconocido" else 0,

        # Internet
        "familia_tiene_internet_No": 1 if tieneinternet == "No" else 0,
        "familia_tiene_internet_Si": 1 if tieneinternet == "Si" else 0,
        #"familia_tiene_internet_Desconocido": 1 if tieneinternet == "Desconocido" else 0,

        # Lavadora
        "familia_tiene_lavadora_No": 1 if tienelavadora == "No" else 0,
        "familia_tiene_lavadora_Si": 1 if tienelavadora == "Si" else 0,
        #"familia_tiene_lavadora_Desconocido": 1 if tienelavadora == "Desconocido" else 0,
    }

    # Formato legible para mostrar en el frontend

    return dummies

def preparar_datos_para_modelo(dummies):

    # Extraer los valores de las dummies en el orden correcto
    dummy_values = list(dummies.values())
    
    # Convertir a un array de numpy
    array_dummies = np.array(dummy_values).reshape(1, -1)  # Forma (1, n_features)
    
    return array_dummies


if __name__ == "__main__":
    app.run_server(debug=True)



In [183]:
resultado = actualizar_dummies(
    ubicacion="URBANO",
    bilingue="S",
    calendario="B",
    caracter="ACADÉMICO",
    jornada="ÚNICA",
    naturaleza="OFICIAL",
    genero="F",
    nacionalidad="COLOMBIA",
    educacionmadre="Educación profesional incompleta",
    educacionpadre="Primaria completa",
    estratovivienda="Estrato 3",
    personashogar="3 a 4",
    tieneautomovil="Si",
    tienecomputador="No",
    tieneinternet="Desconocido",
    tienelavadora="Si"
)

# Mostrar el resultado en la consola
x=preparar_datos_para_modelo(resultado)

x

array([[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 1]])

In [ ]:
modelo = load_model("ruta_al_modelo.h5")
    
# Realizar predicción
#prediccion = modelo.predict(datos_array)